In [1]:

#~ importing libraries

import pandas as pd, numpy as np, requests, os

from meteostat import Hourly, Point
from datetime import datetime

from pathlib import Path

#~ setting up project directories 

PROJECT_ROOT = Path.cwd().resolve().parents[0]

DATA_DIR = PROJECT_ROOT / 'data' 

RAW_DIR = DATA_DIR / 'raw'

PROCESSED_DIR = DATA_DIR / 'processed'

# Generating the Dataset

In [ ]:

#~ fetching hourly weather data from meteostat for Montreal City from 2023-2024
montreal = Point(45.5017, -73.5673)

weather_df = Hourly(
    montreal,
        start=datetime(2023, 1, 1),
            end=datetime(2024, 12, 31)
            ).fetch()


In [3]:

#~ fetching BIXI's station information

URL = "https://gbfs.velobixi.com/gbfs/2-2/en/station_information.json"

raw = requests.get(URL, timeout=30).json()

stations_status_df = pd.json_normalize(raw["data"]["stations"])

keep_stations_columns = [
    "station_id", "name", "lat", "lon", "capacity"
]
stations_status_df = stations_status_df[[c for c in keep_stations_columns if c in stations_status_df.columns]]

In [4]:

bixi_trip_2024_df = pd.read_csv(RAW_DIR/'BIXI_Trip_2024.csv')


bixi_trip_2023_df = pd.read_csv(RAW_DIR/'BIXI_Trip_2023.csv')


bixi_trip_df = pd.concat([bixi_trip_2023_df,bixi_trip_2024_df], ignore_index=True)

# Data Cleaning

In [5]:

'''function for quick description of dataframe'''
def explore_data(df):
    print("Dataframe Shape:", df.shape)
    print("\nDataframe Info:")
    print(df.info())
    print("\nMissing Values:\n", df.isnull().sum())
    print("\nStatistical Summary:\n", df.describe())
    print("\nFirst 5 Rows:\n", df.head())


## BIXI Station Information Dataset

In [6]:
explore_data(stations_status_df)

Dataframe Shape: (239, 5)

Dataframe Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239 entries, 0 to 238
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   station_id  239 non-null    object 
 1   name        239 non-null    object 
 2   lat         239 non-null    float64
 3   lon         239 non-null    float64
 4   capacity    239 non-null    int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 9.5+ KB
None

Missing Values:
 station_id    0
name          0
lat           0
lon           0
capacity      0
dtype: int64

Statistical Summary:
               lat         lon    capacity
count  239.000000  239.000000  239.000000
mean    45.516834  -73.579330   23.719665
std      0.033080    0.027722    9.367882
min     45.446505  -73.667162   11.000000
25%     45.496849  -73.597764   19.000000
50%     45.515868  -73.573338   21.000000
75%     45.537172  -73.562035   27.000000
max     45.630686  -73.494847

In [7]:

#~ sorting the dataframe by station name

stations_status_df = stations_status_df.sort_values(by='name')

stations_status_df['name'].unique().tolist()

['1ere avenue / St-Zotique',
 '20e avenue / de Bellechasse',
 '3e avenue / Bannantyne',
 '3e avenue / Wellington',
 '6e Avenue / Bélanger',
 '9e avenue / Masson',
 'Argyle / Bannantyne',
 'Atwater / Sherbrooke',
 'Aylwin / Ontario',
 'BIXI K5',
 'Beaubien / 8e avenue',
 'Bennett / Ste-Catherine',
 'Benny / Sherbrooke',
 'Bernard / Jeanne-Mance',
 'Berri / Rachel',
 'Beurling / Godin',
 'Bibliothèque et Complexe aquatique Rosemont (8e avenue / Rosemont)',
 'Boileau / Lacordaire',
 'Bourbonnière / du Mont-Royal',
 'Bourret / Victoria',
 'Boyer / Jean-Talon',
 'CHSLD Joseph-François-Perrault (St-Michel / Everett)',
 'CHSLD Robert Cliche (de Bellechasse / 16e avenue)',
 'CHU Ste-Justine (de la Côte Ste-Catherine / Hudson)',
 'Calixa-Lavallée / Rachel',
 'Calixa-Lavallée / Rachel (ouest)',
 'Campus MIL (Outremont / Thérèse-Lavoie Roux)',
 'Cartier / Marie-Anne',
 'Cathcart / McGill College',
 'Chabanel / du Parc',
 'Christophe-Colomb / Beaubien',
 'Clark / Ontario',
 "Clark / de l'Arcade",


## Weather Dataset from MeteoStat API


Link for more info about meteostat:
https://dev.meteostat.net/python/hourly.html#example 

In [8]:
explore_data(weather_df)

Dataframe Shape: (17521, 11)

Dataframe Info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17521 entries, 2023-01-01 00:00:00 to 2024-12-31 00:00:00
Freq: h
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   temp    17521 non-null  Float64
 1   dwpt    17521 non-null  Float64
 2   rhum    17521 non-null  Float64
 3   prcp    17521 non-null  Float64
 4   snow    0 non-null      Float64
 5   wdir    17521 non-null  Float64
 6   wspd    17521 non-null  Float64
 7   wpgt    0 non-null      Float64
 8   pres    17521 non-null  Float64
 9   tsun    0 non-null      Float64
 10  coco    17264 non-null  Float64
dtypes: Float64(11)
memory usage: 1.8 MB
None

Missing Values:
 temp        0
dwpt        0
rhum        0
prcp        0
snow    17521
wdir        0
wspd        0
wpgt    17521
pres        0
tsun    17521
coco      257
dtype: int64

Statistical Summary:
             temp       dwpt       rhum      prcp  snow        wdir 

In [9]:

weather_df.reset_index(inplace=True)

In [10]:

#~ Dropping columns where all values are missing + dropping other not relevant predictors columns

weather_df.drop(columns=['snow','wpgt','tsun', 'pres', 'coco','wdir', 'rhum'], inplace=True)

In [11]:

#~ renaming weather_df's columns to a more descriptive name

weather_df.rename(columns={'temp':'temperature',
                           'dwpt':'dew_point',
                           'prcp':'precipitation',
                           'wspd':'wind_speed'
                           }, inplace=True)

In [12]:

#~ dropping dew_point due to high correlation with temperature

display(weather_df.corr(numeric_only=True))

weather_df.drop(columns=['dew_point'], inplace=True)

,temperature,dew_point,precipitation,wind_speed
temperature,1.000000,0.923112,0.014496,-0.241845
dew_point,0.923112,1.000000,0.088932,-0.253789
precipitation,0.014496,0.088932,1.000000,0.029091
wind_speed,-0.241845,-0.253789,0.029091,1.000000


In [13]:
explore_data(weather_df )

Dataframe Shape: (17521, 4)

Dataframe Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17521 entries, 0 to 17520
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   time           17521 non-null  datetime64[ns]
 1   temperature    17521 non-null  Float64       
 2   precipitation  17521 non-null  Float64       
 3   wind_speed     17521 non-null  Float64       
dtypes: Float64(3), datetime64[ns](1)
memory usage: 599.0 KB
None

Missing Values:
 time             0
temperature      0
precipitation    0
wind_speed       0
dtype: int64

Statistical Summary:
                       time  temperature  precipitation  wind_speed
count                17521      17521.0        17521.0     17521.0
mean   2024-01-01 00:00:00     9.403544       0.135643    5.713823
min    2023-01-01 00:00:00        -28.8            0.0         0.0
25%    2023-07-02 12:00:00          0.8            0.0         3.0
50%    202

## BIXI Trip Dataset

source: https://bixi.com/en/open-data/

In [14]:
explore_data(bixi_trip_df)

Dataframe Shape: (25066166, 10)

Dataframe Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25066166 entries, 0 to 25066165
Data columns (total 10 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   STARTSTATIONNAME            object 
 1   STARTSTATIONARRONDISSEMENT  object 
 2   STARTSTATIONLATITUDE        float64
 3   STARTSTATIONLONGITUDE       float64
 4   ENDSTATIONNAME              object 
 5   ENDSTATIONARRONDISSEMENT    object 
 6   ENDSTATIONLATITUDE          float64
 7   ENDSTATIONLONGITUDE         float64
 8   STARTTIMEMS                 int64  
 9   ENDTIMEMS                   float64
dtypes: float64(5), int64(1), object(4)
memory usage: 1.9+ GB
None

Missing Values:
 STARTSTATIONNAME                5245
STARTSTATIONARRONDISSEMENT      5442
STARTSTATIONLATITUDE            5245
STARTSTATIONLONGITUDE           5245
ENDSTATIONNAME                139918
ENDSTATIONARRONDISSEMENT      141815
ENDSTATIONLATITUDE            139918


In [15]:

#~ standardizing columnsNames of bixi_trip_df

bixi_trip_df.columns = (bixi_trip_df.columns              
              .str.normalize('NFKD')                    
              .str.encode('ascii','ignore').str.decode('ascii')
              .str.strip()
              .str.lower()
              .str.replace(r'[^a-z0-9]+', '_', regex=True)  
              .str.replace(r'_+', '_', regex=True)          
              .str.strip('_'))


it seems that a majority of the null values are missing the endstation name and therefore has no endtime. We will assume that this is error in data capture

We will drop all rows that has null values

In [16]:

bixi_trip_df.dropna(inplace=True)

creating a new column by transforming the startimems and endtimems  (these columns are in UNIX format)
into datetime formatted as YYYY-MM-DD HH-mm-ss

In [17]:
bixi_trip_df['starttime'] = (
    pd.to_datetime(bixi_trip_df["starttimems"], unit='ms', utc=True)
      .dt.floor("s")                           #~ the data showed time zone in ISO-8601 format, I stopped at seconds
      .dt.tz_convert("America/Toronto")
      .dt.tz_localize(None)        
)


bixi_trip_df['endtime'] = (
    pd.to_datetime(bixi_trip_df["endtimems"], unit='ms', utc=True)
      .dt.floor("s")                           
      .dt.tz_convert("America/Toronto")
      .dt.tz_localize(None)       
)

bixi_trip_df.drop(columns=['starttimems', 'endtimems'],inplace=True) #~ dropping the initial unix time columns

In [18]:

#~ adding column to bring out the hour of the day, the day of week, day of month, and month of the year for starting time and endtime

#~Hour of day (0–23)

bixi_trip_df["start_hour"] = bixi_trip_df["starttime"].dt.hour
bixi_trip_df["end_hour"]   = bixi_trip_df["endtime"].dt.hour

#~Day of week (1–7, Monday=1, Sunday=7)

bixi_trip_df["start_day"] = bixi_trip_df["starttime"].dt.dayofweek + 1
bixi_trip_df["end_day"]   = bixi_trip_df["endtime"].dt.dayofweek + 1

#~Day of month (1–31)
bixi_trip_df["start_dayofmonth"] = bixi_trip_df["starttime"].dt.day
bixi_trip_df["end_dayofmonth"]   = bixi_trip_df["endtime"].dt.day

#~Month of year (1–12)

bixi_trip_df["start_month"] = bixi_trip_df["starttime"].dt.month
bixi_trip_df["end_month"]   = bixi_trip_df["endtime"].dt.month

In [21]:
display(bixi_trip_df.head())

,startstationname,startstationarrondissement,startstationlatitude,startstationlongitude,endstationname,endstationarrondissement,endstationlatitude,endstationlongitude,starttime,endtime,start_hour,end_hour,start_day,end_day,start_dayofmonth,end_dayofmonth,start_month,end_month
0,Métro Mont-Royal (Utilités publiques / Rivard),Le Plateau-Mont-Royal,45.524236,-73.581552,Chabot / Mont-Royal,Le Plateau-Mont-Royal,45.534134,-73.573524,2023-10-25 16:44:56,2023-10-25 16:51:32,16,16,3,3,25,25,10,10
1,Rielle / Wellington,Verdun,45.460156,-73.567001,St-Jacques / McGill,Ville-Marie,45.501441,-73.560144,2023-10-25 17:48:55,2023-10-25 18:12:47,17,18,3,3,25,25,10,10
2,Ste-Catherine / Drummond,Ville-Marie,45.498588,-73.574278,Peel / Ottawa,Le Sud-Ouest,45.494200,-73.559985,2023-10-25 22:01:45,2023-10-25 22:06:32,22,22,3,3,25,25,10,10
3,Boyer / du Mont-Royal,Le Plateau-Mont-Royal,45.527432,-73.579917,de l'Hôtel-de-Ville / Rachel,Le Plateau-Mont-Royal,45.519897,-73.580106,2023-10-25 17:31:29,2023-10-25 17:36:54,17,17,3,3,25,25,10,10
4,Clark / Ontario,Ville-Marie,45.510625,-73.566903,Wolfe / Robin,Ville-Marie,45.519581,-73.560116,2023-10-25 17:40:06,2023-10-25 17:47:20,17,17,3,3,25,25,10,10


# Parquet Dataframes

In [20]:

#~ parquet saving cleaned dataframes

weather_df.to_parquet(PROCESSED_DIR/'weather_data.parquet', index=False)

bixi_trip_df.to_parquet(PROCESSED_DIR/'bixi_trip_data.parquet', index=False)

stations_status_df.to_parquet(PROCESSED_DIR/'bixi_stations_data.parquet', index=False)